In [1]:
import csv
import random
from math import log, sqrt

In [2]:
def read_csv(filepath):
    """
    This function reads the example dataset from a CSV file and returns a
    list of lists.
    """
    data = []

    with open(filepath) as fd:
        reader = csv.reader(fd, delimiter=',')

        for row in reader:
            row = [int(row[0]), row[1], row[2], int(row[3]), row[4],
                   row[5], row[6], row[7], row[8], int(row[9]),
                   int(row[10]), int(row[11]), row[12], row[13]]
            data.append(list(map(lambda x: x.strip() if isinstance(x, str) else x, row)))

    return data

In [3]:
class DecisionTreeClassifier:

    class DecisionNode:
        def __init__(self, col=-1, value=None, results=None, tb=None, fb=None):
            self.col = col
            self.value = value
            self.results = results
            self.tb = tb
            self.fb = fb

    """
    :param  max_depth:          numero maximo de quebras durante o treinamento
    :param  random_features:    Se estiver como Falso, todas a variaveis 
                                vao ser usadas para treino e predicao, 
                                senao sera escolhido um conjunto aleatorio 
                                de tamanho sqrt(nb features)
    """
    def __init__(self, max_depth=-1, random_features=False):
        self.root_node = None
        self.max_depth = max_depth
        self.features_indexes = []
        self.random_features = random_features

    def fit(self, rows, criterion=None):
        if len(rows) < 1:
            raise ValueError("Nao ha amostras suficientes no dataset de entrada.")

        if not criterion:
            criterion = self.entropy
        if self.random_features:
            self.features_indexes = self.choose_random_features(rows[0])
            rows = [self.get_features_subset(row) + [row[-1]] for row in rows]

        self.root_node = self.build_tree(rows, criterion, self.max_depth)

    """
    Retorna uma predicao para as variaveis passadas como parametro
    :param  features:   uma lista de valores
    """
    def predict(self, features):
        if self.random_features:
            if not all(i in range(len(features))
                       for i in self.features_indexes):
                raise ValueError("As variaveis passadas nao batem com o conjunto utilizado para treino")
            features = self.get_features_subset(features)

        return self.classify(features, self.root_node)

    """
    Seleciona indices aleatoriamente
    """
    def choose_random_features(self, row):
        nb_features = len(row) - 1
        return random.sample(range(nb_features), int(sqrt(nb_features)))

    """
    Retorna os valores selecionado randomicamente
    """
    def get_features_subset(self, row):
        return [row[i] for i in self.features_indexes]

    """
    Divide o dataset de acordo com o valor do indice da coluna
    :param  rows: o dataset
    :param  column: o indice da coluna usada para quebrar o dado
    :param  value: o valor usado para a quebra
    """
    def divide_set(self, rows, column, value):
        split_function = None
        if isinstance(value, int) or isinstance(value, float):
            split_function = lambda row: row[column] >= value
        else:
            split_function = lambda row: row[column] == value

        set1 = [row for row in rows if split_function(row)]
        set2 = [row for row in rows if not split_function(row)]

        return set1, set2

    """
    Retorna a ocorrencia de cada resultado
    """
    def unique_counts(self, rows):
        results = {}
        for row in rows:
            r = row[len(row) - 1]
            if r not in results:
                results[r] = 0
            results[r] += 1
        return results

    """
    Retorna a entropia das linhas passada como parametro (lista de listas)
    """
    def entropy(self, rows):
        log2 = lambda x: log(x) / log(2)
        results = self.unique_counts(rows)
        ent = 0.0
        for r in results.keys():
            p = float(results[r]) / len(rows)
            ent = ent - p * log2(p)
        return ent

    """
    Cria recursivamente a arvore de decisao quebrando o dataset ate que nenhum
    ganho de informacao seja adicionado, ou ate que se chegue no maxmio de profundidade
    """
    def build_tree(self, rows, func, depth):
        if len(rows) == 0:
            return self.DecisionNode()
        if depth == 0:
            return self.DecisionNode(results=self.unique_counts(rows))

        current_score = func(rows)
        best_gain = 0.0
        best_criteria = None
        best_sets = None
        column_count = len(rows[0]) - 1

        for col in range(0, column_count):
            column_values = {}
            for row in rows:
                column_values[row[col]] = 1
            for value in column_values.keys():
                set1, set2 = self.divide_set(rows, col, value)

                p = float(len(set1)) / len(rows)
                gain = current_score - p * func(set1) - (1 - p) * func(set2)
                if gain > best_gain and len(set1) > 0 and len(set2) > 0:
                    best_gain = gain
                    best_criteria = (col, value)
                    best_sets = (set1, set2)

        if best_gain > 0:
            trueBranch = self.build_tree(best_sets[0], func, depth - 1)
            falseBranch = self.build_tree(best_sets[1], func, depth - 1)
            return self.DecisionNode(col=best_criteria[0],
                                     value=best_criteria[1],
                                     tb=trueBranch, fb=falseBranch)
        else:
            return self.DecisionNode(results=self.unique_counts(rows))

    """
    Faz uma predicao usando as variavies passadas como parametro
    """
    def classify(self, observation, tree):
        if tree.results is not None:
            return list(tree.results.keys())[0]
        else:
            v = observation[tree.col]
            branch = None
            if isinstance(v, int) or isinstance(v, float):
                if v >= tree.value:
                    branch = tree.tb
                else:
                    branch = tree.fb
            else:
                if v == tree.value:
                    branch = tree.tb
                else:
                    branch = tree.fb
            return self.classify(observation, branch)

In [4]:
def test_tree():
    data = read_csv("../data/income.csv")
    tree = DecisionTreeClassifier(random_features=True)
    tree.fit(data)

    print(tree.predict([39, 'State-gov', 'Bachelors', 13, 'Never-married',
                        'Adm-clerical', 'Not-in-family', 'White', 'Male',
                        2174, 0, 40, 'United-States']))


if __name__ == '__main__':
    test_tree()

<=50K


In [5]:
import logging
import random
from concurrent.futures import ProcessPoolExecutor

In [6]:
class RandomForestClassifier(object):

    """
    :param  num_trees: Numero de arvores de decisao
    :param  num_samples: Numero de amostras para cada arvore
    :param  max_depth: Profundidade maxima da arvore
    :param  max_workers: Numero maximo de processos para treinamento
    """
    def __init__(self, num_trees, num_samples, max_depth=-1, max_workers=1):
        self.trees = []
        self.num_trees = num_trees
        self.num_samples = num_samples
        self.max_depth = max_depth
        self.max_workers = max_workers

    def fit(self, data):
        with ProcessPoolExecutor(max_workers=self.max_workers) as executor:
            rand_fts = map(lambda x: [x, random.sample(data, self.num_samples)],range(self.num_trees))
            self.trees = list(executor.map(self.train_tree, rand_fts))

    """
    Treina uma unica arvore e a retorna
    """
    def train_tree(self, data):
        logging.info('Training tree {}'.format(data[0] + 1))
        tree = DecisionTreeClassifier(max_depth=self.max_depth)
        tree.fit(data[1])
        return tree

    def predict(self, feature):
        predictions = []

        for tree in self.trees:
            predictions.append(tree.predict(feature))

        return max(set(predictions), key=predictions.count)

In [7]:
def test_rf():
    from sklearn.model_selection import train_test_split

    data = read_csv("../data/income.csv")
    train, test = train_test_split(data, test_size=0.3)

    #rf = RandomForestClassifier(num_trees=60, num_samples=3000, max_workers=4)
    rf = RandomForestClassifier(num_trees=60, num_samples=600, max_workers=1)
    rf.fit(train)

    errors = 0
    features = [ft[:-1] for ft in test]
    values = [ft[-1] for ft in test]

    for feature, value in zip(features, values):
        prediction = rf.predict(feature)
        if prediction != value:
            errors += 1

    logging.info("Error rate: {}".format(errors / len(features) * 100))


if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    from timeit import default_timer as timer

    start = timer()
    
    test_rf()
    
    end = timer()
    print('Execution took: %s secs' % (end - start))

INFO:root:Training tree 1
INFO:root:Training tree 2
INFO:root:Training tree 3
INFO:root:Training tree 4
INFO:root:Training tree 5
INFO:root:Training tree 6
INFO:root:Training tree 7
INFO:root:Training tree 8
INFO:root:Training tree 9
INFO:root:Training tree 10
INFO:root:Training tree 11
INFO:root:Training tree 12
INFO:root:Training tree 13
INFO:root:Training tree 14
INFO:root:Training tree 15
INFO:root:Training tree 16
INFO:root:Training tree 17
INFO:root:Training tree 18
INFO:root:Training tree 19
INFO:root:Training tree 20
INFO:root:Training tree 21
INFO:root:Training tree 22
INFO:root:Training tree 23
INFO:root:Training tree 24
INFO:root:Training tree 25
INFO:root:Training tree 26
INFO:root:Training tree 27
INFO:root:Training tree 28
INFO:root:Training tree 29
INFO:root:Training tree 30
INFO:root:Training tree 31
INFO:root:Training tree 32
INFO:root:Training tree 33
INFO:root:Training tree 34
INFO:root:Training tree 35
INFO:root:Training tree 36
INFO:root:Training tree 37
INFO:root:

Execution took: 43.118200434000755 secs
